In [39]:
import psycopg2
import requests
from psycopg2 import sql
from datetime import datetime, timedelta
import pandas as pd
DB_HOST = 'localhost'
DB_NAME = 'tw'
DB_USER = 'mini'
DB_PASS = 'buddyrich134'
def get_db_connection():
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
    return conn

conn = get_db_connection()
cursor = conn.cursor()

In [2]:
da_newest = rows[0]
final_list = []
for i in range((datetime.now() - da_newest).days):
    try:
        da = da_newest + timedelta(i+1)
        year, mo, date = da.year, da.month, da.day
        mo = f"0{mo}" if mo < 10 else mo
        date = f"0{date}" if date < 10 else date
        print(year, mo, date)
        url = f'https://www.twse.com.tw/rwd/zh/block/BFIAUU?date={year}{mo}{date}&selectType=S&response=json'
        print(url)
        res = requests.get(url)
        res.raise_for_status()
        data = res.json().get('data', [])
        da = res.json()['title'].replace("年", "-").replace("月", "-")[:9]
        year = str(int(da[:3]) + 1911)
        da = year + da[3:]
        if not data: continue
        data = [i + [da] for i in data[:-1]]
        for j in data:
            final_list.append(j)
        df = pd.DataFrame(final_list)
        df[3] = df[3].str.replace(',', '')
        df[3] = df[3].apply(lambda x: float(x) if x != '' else 0.0)
        df[4] = df[4].str.replace(',', '').astype(int)
        df[5] = df[5].str.replace(',', '').astype(int)
        df[6] = pd.to_datetime(df[6], format='%Y-%m-%d') 
        
        df.columns = ['code', 'codename', 'type_', 'cl', 'vol_share', 'vol', 'da']
        df = df.drop_duplicates(subset=['code', 'da'])
    except NameError:
        print("no data to update, passed")

2024 08 10
https://www.twse.com.tw/rwd/zh/block/BFIAUU?date=20240810&selectType=S&response=json
2024 08 11
https://www.twse.com.tw/rwd/zh/block/BFIAUU?date=20240811&selectType=S&response=json


NameError: name 'df' is not defined

In [9]:
df = yf.download(f"00687B.TWO", start='2024-08-01', progress=False)
df


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-08-01,243.100006,243.100006,238.600006,242.000000,242.000000,7249228
2024-08-02,227.649994,230.199997,219.000000,219.000000,219.000000,15423808
2024-08-05,195.750000,195.750000,176.899994,176.899994,176.899994,32296502
2024-08-06,195.399994,199.800003,178.550003,194.000000,194.000000,29414186
2024-08-07,197.500000,210.199997,196.100006,209.300003,209.300003,14207884
2024-08-08,201.100006,205.500000,196.850006,200.350006,200.350006,12821576
2024-08-09,211.050003,215.050003,208.399994,211.699997,211.699997,13335516


In [ ]:
url_stock_list_goodinfo = "https://goodinfo.tw/tw/Lib.js/StockTW_ID_NM_List.js"
res = requests.get(url_stock_list_goodinfo)
soup = BeautifulSoup(res.text, 'lxml')
list_ = soup.string.split("','")

In [33]:
from bs4 import BeautifulSoup
import requests
import yfinance as yf
import json
from tqdm import tqdm
url_stock_list_goodinfo = "https://goodinfo.tw/tw/Lib.js/StockTW_ID_NM_List.js"
res = requests.get(url_stock_list_goodinfo)
soup = BeautifulSoup(res.text, 'lxml')
list_ = soup.string.split("','")
list_4 = [i.split(" ")[0] for i in list_1[2].split("','")[2:]][:2179]
list_4[-1] = list_4[-1].split("'")[0]
list_4_cname = [i.split(" ")[0] for i in list_1[3].split("','")[2:]][:2179]
list_4_cname[-1] = list_4_cname[-1].split("'")[0]

list_listed = []
for ticker in tqdm(list_4):
    df = yf.download(f"{ticker}.TW", start='2024-08-01', progress=False)
    if len(df) != 0:
       list_listed.append("TW")
    else:
        df = yf.download(f"{ticker}.TWO", start='2024-08-01', progress=False)
        if len(df) != 0:
            list_listed.append("TWO")

        else:
            list_listed.append("None")
df = pd.DataFrame({
    "code": list_4,
    "cname": list_4_cname,
    "listed": list_listed
})

  0%|          | 7/2179 [00:00<03:47,  9.53it/s]
1 Failed download:
['006201.TW']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-08-01 -> 2024-08-11)')
  0%|          | 9/2179 [00:01<05:43,  6.31it/s]

$006201.TW: possibly delisted; No price data found  (1d 2024-08-01 -> 2024-08-11)


  3%|▎         | 61/2179 [00:11<07:26,  4.75it/s]
1 Failed download:
['00679B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  3%|▎         | 69/2179 [00:14<06:59,  5.03it/s]
1 Failed download:
['00687B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  3%|▎         | 75/2179 [00:16<07:31,  4.66it/s]
1 Failed download:
['00694B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  3%|▎         | 76/2179 [00:16<13:14,  2.65it/s]
1 Failed download:
['00695B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  4%|▎         | 77/2179 [00:17<19:27,  1.80it/s]
1 Failed download:
['00696B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  4%|▎         | 78/2179 [00:18<22:25,  1.56it/s]
1 Failed download:
['00697B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  4%|▍         | 94/2179 [00:22<08:45,  3.97it/s]
1 Failed download:
['00718B.TW']: YFTz

$00838B.TWO: possibly delisted; No price data found  (1d 2024-08-01 -> 2024-08-11)



1 Failed download:
['00840B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  8%|▊         | 164/2179 [01:11<24:25,  1.38it/s]
1 Failed download:
['00841B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  8%|▊         | 165/2179 [01:12<24:33,  1.37it/s]
1 Failed download:
['00842B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  8%|▊         | 166/2179 [01:13<24:41,  1.36it/s]
1 Failed download:
['00844B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  8%|▊         | 167/2179 [01:13<25:34,  1.31it/s]
1 Failed download:
['00845B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  8%|▊         | 168/2179 [01:14<25:08,  1.33it/s]
1 Failed download:
['00846B.TW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
  8%|▊         | 169/2179 [01:15<26:11,  1.28it/s]
1 Failed download:
['00847B.TW']: YFTzMissingError('$%ticker%: possibly delisted;

In [40]:

cursor.executemany(f'''
INSERT INTO public.maincode (code, cname, listed)
VALUES (%s, %s, %s)
''', df.values.tolist())
conn.commit()

In [74]:
[i[0] for i in res]

['0050',
 '0051',
 '0052',
 '0053',
 '0055',
 '0056',
 '0057',
 '0061',
 '006203',
 '006204',
 '006205',
 '006206',
 '006207',
 '006208',
 '00625K',
 '00631L',
 '00632R',
 '00633L',
 '00634R',
 '00635U',
 '00636',
 '00636K',
 '00637L',
 '00638R',
 '00639',
 '00640L',
 '00641R',
 '00642U',
 '00643',
 '00643K',
 '00645',
 '00646',
 '00647L',
 '00648R',
 '00650L',
 '00651R',
 '00652',
 '00653L',
 '00654R',
 '00655L',
 '00656R',
 '00657',
 '00657K',
 '00660',
 '00661',
 '00662',
 '00663L',
 '00664R',
 '00665L',
 '00666R',
 '00668',
 '00668K',
 '00669R',
 '00670L',
 '00671R',
 '00673R',
 '00674R',
 '00675L',
 '00676R',
 '00678',
 '00680L',
 '00681R',
 '00682U',
 '00683L',
 '00684R',
 '00685L',
 '00686R',
 '00688L',
 '00689R',
 '00690',
 '00692',
 '00693U',
 '00700',
 '00701',
 '00702',
 '00703',
 '00706L',
 '00707R',
 '00708L',
 '00709',
 '00710B',
 '00711B',
 '00712',
 '00713',
 '00714',
 '00715L',
 '00717',
 '00728',
 '00730',
 '00731',
 '00733',
 '00735',
 '00736',
 '00737',
 '00738U',
 

In [92]:
df_list = []
for year in range(2024, 2023, -1):
    starter = pd.to_datetime('2024-01-01')
    for da in tqdm(range(0, 366)):
        date = f"0{da}" if da < 0 else da
        try:
            url_json = f'https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date={year}01{date}&selectType=ALL&response=json'
            response = requests.get(url_json)
            data = response.json()['data']
            data = [i + [starter] for i in data]
            df_list.append(data)
        except KeyError as e:
            pass
        starter += timedelta(1)
    df = pd.DataFrame(sum(df_list, []), columns=['code', 'cname', 'yield', 'year', 'pe', 'pb', 'financial_year', 'da'])


 30%|███       | 3/10 [00:00<00:00, 21.15it/s]

2024-01-01 00:00:00
2024-01-02 00:00:00
2024-01-03 00:00:00
2024-01-04 00:00:00
2024-01-05 00:00:00


 60%|██████    | 6/10 [00:00<00:00, 18.63it/s]

2024-01-06 00:00:00
2024-01-07 00:00:00
2024-01-08 00:00:00
2024-01-09 00:00:00


100%|██████████| 10/10 [00:00<00:00, 18.19it/s]

2024-01-10 00:00:00


In [123]:
year = 2024
list_ = []
query = sql.SQL("select da from price where code = 'TWSE Index' order by da desc limit 1;")
cursor.execute(query)
rows = cursor.fetchone()
da_newest = rows[0]
newest_mo = da_newest.month
current_mo = datetime.now().month
for mo in range(current_mo - newest_mo + 1):
    month = f"0{current_mo}" if current_mo < 10 else current_mo
    url_twse = f"https://www.twse.com.tw/rwd/zh/TAIEX/MI_5MINS_HIST?date={year}{month}01&response=json"
    res = requests.get(url_twse)
    js = res.json()
    list_.append(js['data'])
    current_mo += 1
list_list = sum(list_, [])
df = pd.DataFrame(list_list, columns=['da', 'op', 'hi', 'lo', 'cl'])
print(df)
def convert_to_2024(da):
    year, month, day = da.split("/")
    return pd.to_datetime(f"{int(year)+1911}-{month}-{day}")
df['da'] = df['da'].apply(convert_to_2024)
for i in range(1, len(df.columns)):
    df.iloc[:, i] = df.iloc[:, i].apply(lambda x: float(x.replace(",", "")))
print(df)
appending =df[df['da'] > da_newest]
if len(appending) == 0:
    print("nothing to append")
else:
    self.insert_df_into_db(table='public.price')
    

          da         op         hi         lo         cl
0  113/08/01  22,546.40  22,652.21  22,465.06  22,642.10
1  113/08/02  22,141.20  22,141.20  21,636.22  21,638.09
2  113/08/05  20,982.92  20,982.92  19,783.50  19,830.88
3  113/08/06  20,344.39  20,751.50  19,662.74  20,501.02
4  113/08/07  20,638.28  21,306.23  20,638.28  21,295.28
5  113/08/08  21,047.16  21,074.19  20,723.26  20,870.10
6  113/08/09  21,212.00  21,613.76  21,212.00  21,469.00
          da        op        hi        lo        cl
0 2024-08-01   22546.4  22652.21  22465.06   22642.1
1 2024-08-02   22141.2   22141.2  21636.22  21638.09
2 2024-08-05  20982.92  20982.92   19783.5  19830.88
3 2024-08-06  20344.39   20751.5  19662.74  20501.02
4 2024-08-07  20638.28  21306.23  20638.28  21295.28
5 2024-08-08  21047.16  21074.19  20723.26   20870.1
6 2024-08-09   21212.0  21613.76   21212.0   21469.0


,da,op,hi,lo,cl


In [113]:
da_newest - timedelta(10)

datetime.datetime(2024, 7, 30, 0, 0)

In [97]:
import pandas as pd

# Example DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

# Insert a new column 'C' at position 1
df.insert(loc=1, column='C', value=[7, 8, 9])

print(df)


   A  C  B
0  1  7  4
1  2  8  5
2  3  9  6


In [86]:
pd.concat(df_list)

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [69]:
cursor.execute(sql.SQL("Select code from public.maincode where listed = 'TW';"))
conn.commit()
res = cursor.fetchall()
res[0][0]

'0050'

In [64]:
cursor.execute(sql.SQL("Select code from public.maincode where listed = 'TW';"))
conn.commit()
res = cursor.fetchall()

def _(code):
    list_concat = []
    for year in range(self.year, 2016, -1):
        limit_month = 9 if year == self.year else 13
        for index, month in enumerate(range(1, limit_month)):
            try:
                month = f"0{month}" if month < 10 else month
                da = f"{year}{month}01"
                url_json = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={da}&stockNo={code}"
                response = requests.get(url_json)
                list_concat.append(response.json()['data'])
            except:
                if month == 1:
                    return None
                else:
                    list_concat = [item for item in list_concat if item is not None]                                        
                    break
    df_final = pd.DataFrame(sum(list_concat, []))
    df_final = df_final.iloc[:, 0:-2]
    df_final.columns = ['da', 'vol_share', 'vol', 'op', 'hi', 'lo', 'cl']
    df_final.replace("--", None, inplace=True)
    df_final.replace("-", None, inplace=True)
    df_final.ffill(inplace=True)
    df_final['code'] = str(code)
    def convert_to_2024(da):
        year, month, day = da.split("/")
        return f"{int(year)+1911}-{month}-{day}"
    df_final['da'] = df_final['da'].apply(convert_to_2024)
    for i in range(1, len(df_final.columns)):
        df_final.iloc[:, i] = df_final.iloc[:, i].apply(lambda x: float(x.replace(",", "")))
    df_final['vol_share'] = df_final['vol_share'].astype(int) 
    df_final['vol'] = df_final['vol'].astype(int) 
    return df_final 

df_list = []
for i in tqdm(res[:10]):
    code = i[0]
    df = _(code)
    self.insert

100%|██████████| 10/10 [00:39<00:00,  3.93s/it]


In [66]:
pd.concat(df_list)

,da,vol_share,vol,op,hi,lo,cl,code
0,2024-01-02,6283610.0,848793104.0,135.6,135.95,134.65,134.9,50.0
1,2024-01-03,14783894.0,1963126583.0,133.7,133.9,132.3,132.55,50.0
2,2024-01-04,4838660.0,641202865.0,132.55,132.75,132.3,132.5,50.0
3,2024-01-05,4566320.0,604475455.0,132.55,132.95,132.1,132.15,50.0
4,2024-01-08,9657556.0,1285524342.0,133.0,133.6,132.75,132.75,50.0
...,...,...,...,...,...,...,...,...
1849,2017-12-25,9000.0,470250.0,52.25,52.25,52.25,52.25,6204.0
1850,2017-12-26,17000.0,881900.0,52.3,52.3,51.85,51.85,6204.0
1851,2017-12-27,13000.0,675400.0,51.95,52.0,51.95,52.0,6204.0
1852,2017-12-28,12000.0,630050.0,52.55,52.55,52.5,52.5,6204.0


In [62]:
df1 = df.sort_values(by='da', ascending=True)
df1['vol'][0]

16549619798.0

In [ ]:
"日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"